# Building cancer-detection system with RadIO

## 1. CT-scans and Lung-cancer

In [1]:
# some imports
from ipywidgets import interact
from plotting_tools import pil_plot_slices
from example import load_example
import os

* CT-scans are 3d-arrays of data. Cancer in lungs is represented in *pulmonary nodules*:

In [2]:
# plot the scan and corresponding mask
PATH_TO_SAMPLE = '/home/alexander/Work/Temp/unrarred/needed/other/_demo_data/scans_sample/'
bch = load_example(path=os.path.join(PATH_TO_SAMPLE,
                                     '1.3.6.1.4.1.14519.5.2.1.6279.6001.621916089407825046337959219998'))
num_item = 0
interact(lambda height: pil_plot_slices(height, bch.get(num_item, 'images'), bch.get(num_item, 'masks')),
         height=(0, 0.99, 0.01))

interactive(children=(FloatSlider(value=0.49, description='height', max=0.99, step=0.01), Output()), _dom_clas…

<function __main__.<lambda>(height)>

## 2. Dealing with imperfect annotation

## 3. Building detection system-workflow

### `Dataset` and `Pipeline`

CT-scans are voluminous. It is not possible to store all scans from the dataset in memory at once. In order to work with the dataset of scans, we create `Dataset`: an *indexing strructure*:

In [3]:
import sys
sys.path.append('..')

In [10]:
from radio.dataset import FilesIndex, Dataset, Pipeline
from radio import CTImagesMaskedBatch

luna_index = FilesIndex(path=os.path.join(PATH_TO_SAMPLE, '*'),
                        dirs=True)                                    # preparing indexing structure
lunaset = Dataset(index=luna_index, batch_class=CTImagesMaskedBatch)  # Dataset: logic + indexing structure

In [11]:
luna_index.indices[:3]

array(['1.3.6.1.4.1.14519.5.2.1.6279.6001.240969450540588211676803094518',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.837810280808122125183730411210',
       '1.3.6.1.4.1.14519.5.2.1.6279.6001.197063290812663596858124411210'],
      dtype='<U136')

In [12]:
luna_index.get_fullpath(luna_index.indices[0])

'/home/alexander/Work/Temp/unrarred/needed/other/_demo_data/scans_sample/1.3.6.1.4.1.14519.5.2.1.6279.6001.240969450540588211676803094518'

Data-processing workflows are called `pipelines`, which represent sequences(plans) of actions on data

In [ ]:
# do not run it, it's fake!
some_pipeline = (Pipeline()
                 .load()
                 .preprocess_somehow()
                 .do_something_else()
                 ....)

To perform some real computation we need to pass a batch through a `pipeline`:

In [ ]:
# do not run it, it's fake!
batch = (dataset >> some_pipeline).next_batch(3)

### 3.1 Preprocessing

First things first: load the data. `RadIO` uses asychronous loading with `aiofiles`. Should be fast!

In [37]:
preprocessing = (Pipeline()
                 .load(fmt='blosc', components=['images', 'spacing', 'origin']))

Luna-dataset provides cancer-annotations for targets:

In [38]:
ANNO_PATH = '/home/alexander/Work/Data/CSVFILES/annotations.csv'

In [39]:
import pandas as pd
nodules_df = pd.read_csv(ANNO_PATH)
nodules_df.head(2)

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708


Make use of it!

In [40]:
preprocessing += (Pipeline()
                  .fetch_nodules_info(nodules_df)
                  .create_mask())

Normalize data-range to [0, 255]

In [41]:
preprocessing += (Pipeline()
                  .normalize_hu())

Perhaps you need to `dump` preprocessed scans and get back to them later?

In [42]:
preprocessing += (Pipeline()
                  .dump(fmt='blosc', dst='/home/alexander/Work/Data/Temp_dump/',
                        components=['images', 'masks', 'spacing', 'origin'], i8_encoding_mode='linear'))

In [53]:
batch = (lunaset >> preprocessing).next_batch(1)

In [51]:
sys.path.append('../tutorials/')

In [55]:
from utils import show_slices
import numpy as np
ixs = np.array(['1.3.6.1.4.1.14519.5.2.1.6279.6001.768276876111112560631432843476',
                '1.3.6.1.4.1.14519.5.2.1.6279.6001.621916089407825046337959219998'])


two_scans_dataset = Dataset(index=luna_index.create_subset(ixs), batch_class=CTImagesMaskedBatch)
batch = (two_scans_dataset >> preprocessing).next_batch(2, shuffle=False)   # pass a batch through the workflow
show_slices(batch, scan_indices=[0, 1], ns_slice=[211, 67], grid=True, clims=(0, 255))

One can see that the lungs are larger on the left scan. This can be explained by a big difference in scale (represented by a grid). This can be a source of uncontrollable variation in the training dataset. In order to make scans more isotropic, we add action `unify_spacing` to our preprocessing workflow.

In [48]:
SHAPE = (256, 256, 256)
SPACING = (1.7, 1., 1.)

In [49]:
preprocessing += (Pipeline()
                  .unify_spacing(spacing=SPACING, shape=SHAPE))

Now pass a batch through the workflow and see the difference:

In [ ]:
batch = (two_scans_dataset >> preprocessing).next_batch(2)

In [ ]:
show_slices(batch, scan_indices=[0, 1], ns_slice=[124, 124], grid=True, clims=(0, 255))

### 3.2 Augmentation

Let us add some augmentation actions to the workflow. To begin with, let's *controllably* introduce variation into the scale by randomly zooming in/zooming out on scans:

In [74]:
spacing_options = [(1.7, 0.5, 0.5), (1.7, 1.0, 1.0), (1.7, 1.5, 1.5), (1.7, 2.0, 2.0)]

here we define spacing randomizer, a function that randomly fetches spacing:

In [75]:
spacing_randomizer = lambda *args: spacing_options[np.random.choice(range(len(spacing_options)))]

In [76]:
from radio.dataset import F                                          # see below the explanation
augmentation = (Pipeline()                                           # empty workflow
                .load(fmt='raw')                                     # load scans
                .unify_spacing(spacing=F(spacing_randomizer), shape=(256, 256, 256)))

add some rotation:

In [77]:
augmentation += (Pipeline()
                 .rotate(random=True, angle=30, components=['images', 'masks']))

### MIPs: a way to reduce 3d-problem to a 2d-one

In [79]:
xips = batch.xip('images', mode='max', depth=7, stride=3)

In [80]:
lxip = int(len(xips) / len(batch))

In [82]:
interact(lambda height: pil_plot_slices(height, batch.get(num_item, 'images'), xips[0:lxip, :, :, 0],
                                        batch.get(num_item, 'masks')),
         height=(0, 0.99, 0.005))

interactive(children=(FloatSlider(value=0.495, description='height', max=0.99, step=0.005), Output()), _dom_cl…

<function __main__.<lambda>(height)>

### 3.3 Model training

In [ ]:
import tensorflow as tf
from radio.dataset import B, V, F
from radio.dataset.models.tf import TFModel

model_path = 'unet/e_6_3c_unet/models/logloss/'
config = dict(load=dict(path=model_path),
              session=dict(config=tf.ConfigProto(allow_soft_placement=True)))


SHAPE = (128, 256, 256)
SPACING = (1.7, 1, 1)
XIP_PARAMS = dict(mode='max', depth=6, stride=2, channels=3)

ppl_predict_scan = (Pipeline()
                    .init_model('static', TFModel, 'xipnet', config)
                    .load(fmt='blosc', components=['images', 'spacing', 'origin', 'masks'])
                    .fetch_nodules_from_mask()
                    .init_variables(['predictions', 'nodules_true', 'nodules_predicted'])
                    .fetch_nodules_from_mask()
                    .update_variable('nodules_true', B('nodules'))
                    .predict_model('xipnet', save_to=V('predictions'),
                                   feed_dict=dict(images=F(CTIMB.xip, component='images', **XIP_PARAMS)))
                    .call(CTIMB.unxip, xip=V('predictions'), squeeze=True, **XIP_PARAMS,
                          component='masks')
                    .fetch_nodules_from_mask()
                    .update_variable('nodules_predicted', B('nodules')) << lunaset)
